## Loading Vector Store

In [1]:
from settings import *

In [3]:
# Loading the vector_store
from langchain_community.vectorstores import FAISS
from utils import LlamaCppEmbeddingsFix

embeding = LlamaCppEmbeddingsFix(
    model_path=MODEL_PATH, verbose=False,
    n_ctx=(N_DOCS_RETREIVE+2)*CHUNK_SIZE,
)

vector_store = FAISS.load_local(
    VECTOR_STORE,
    embeddings=embeding,
    allow_dangerous_deserialization=True
)
vector_store.load_local(VECTOR_STORE, embeddings=embeding, allow_dangerous_deserialization=True)
print(f"Loaded {vector_store.index.ntotal} Documents")

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Loaded 20 Documents


In [4]:
SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Context: {context}:"""

In [5]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={
    'k': N_DOCS_RETREIVE
})

In [8]:
from langchain_community.chat_models import ChatLlamaCpp

llm = ChatLlamaCpp(
    temperature=0.2,
    model_path=MODEL_PATH,
    n_ctx=(N_DOCS_RETREIVE+2)*CHUNK_SIZE,
    n_batch=32,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=4098,
    top_p=1,
    verbose=False
)

llama_new_context_with_model: n_ctx_per_seq (7008) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [15]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
#QUERY = "How to update followup status"
#QUERY = "How to contact Vishal Pathak"
QUERY = "How to assign fee structure" # Creating Hellucination

docs = retriever.invoke(QUERY)
docs_text = "".join(f"Page {d.metadata['page']}\n"+d.page_content for d in docs)

llm.invoke(
    [
        SystemMessage(SYSTEM_PROMPT.format(context=docs_text)), 
        HumanMessage(QUERY)
    ]
).content

'According to the Operational Manual, assigning a fee structure involves the following steps:\n\n1. **Define the purpose**: Determine why you want to assign a fee structure (e.g., for courses, exams, or other activities).\n2. **Identify the stakeholders**: Who will be affected by the assignment of a fee structure? This includes students, faculty members, staff, and any other relevant parties.\n3. **Determine the fees**: Based on your research and analysis, determine the fees that you want to assign to each stakeholder group.\n4. **Create a fee structure template**: Create a template that outlines the fees for each stakeholder group. This template should include columns for:\n\t* Fee type (e.g., tuition, exam fees)\n\t* Fee amount\n\t* Payment terms (e.g., due date, payment methods)\n5. **Review and revise the fee structure**: Review your fee structure to ensure that it is fair, transparent, and compliant with relevant laws and regulations.\n6. **Communicate the fee structure to stakeho

[Document(metadata={'source': 'data/Manual Administration.pdf', 'page': 0, 'start_index': 0}, page_content='For more information, please contact\n+917970001117, +917970001018\nor\nVisit www.qlsacademy.com\nOPERATIONAL MANUAL\nExplore the world of future learning through AI-based online interactive class\nClick here to download app\nLast Update\n 17-Jan-21\nFOR INSTITUTIONAL LMS OPERATIONS'),
 Document(metadata={'source': 'data/Manual Administration.pdf', 'page': 7, 'start_index': 0}, page_content='Step-1. Click on this icon to check follow-ups History.\nStep-2. Follow-ups History.\n1\n2\n1\nStep-1. Click on Three Dots & Select Follow Up option. \nStep-2. Select Follow-up activity date.\nStep-3. Enter Activity Remark.\nStep-4. Tick Next Follow-up Check Box.\nStep-5. Select Next Action date & Next Action time.\nStep-6. Write next follow-up plans then click on Submit.\nCHECK FOLLOWUP HISTORY\nENTER NEW FOLLOWUP DETAILS\n2\n3\n5 6\n4'),
 Document(metadata={'source': 'data/Manual Administra